In [1]:
import sys
sys.path.append(r'F:/道路拥堵情况预测/DLinear')

import argparse
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from util.data_factory import data_provider
from util.tools import adjust_learning_rate
# params
from layers import DLinear
import matplotlib.pyplot as plt

In [2]:
# 搭建模型Dlinear模型
class SCINetinitialization():
    def __init__(self, args):
        super(SCINetinitialization, self).__init__()
        self.args = args
        self.model, self.device = self.build_model(args)

    def build_model(self, args):
        model = DLinear.Model(self.args).float()
        # 将模型定义在GPU上

        if args.use_gpu:
            device = torch.device('cuda:{}'.format(self.args.device))
            print('Use GPU: cuda:{}'.format(args.device))
        else:
            print('Use CPU')
            device = torch.device('cpu')
        total = sum([param.nelement() for param in model.parameters()])
        print('Number of parameters: %.2fK' % (total / 1e3))  # 打印模型参数量

        if self.args.use_gpu:
            model = nn.DataParallel(model, device_ids=[device])

        return model, device

    def _get_data(self, flag, pre_data=None):
        data_set, data_loader = data_provider(self.args, flag, pre_data)
        return data_set, data_loader

    def train(self, setting):
        train_data, train_loader = self._get_data(flag='train')

        path = os.path.join(self.args.checkpoints, setting)
        if not os.path.exists(path):
            os.makedirs(path)

        time_now = time.time()
        train_steps = len(train_loader)

        model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate)
        criterion = nn.MSELoss()

        for epoch in range(self.args.train_epochs):
            iter_count = 0
            train_loss = []
            self.model.train()
            epoch_time = time.time()
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                model_optim.zero_grad()

                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float().to(self.device)
                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)

                outputs = self.model(batch_x)

                f_dim = -1 if self.args.features == 'MS' else 0
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                loss = criterion(outputs, batch_y)
                train_loss.append(loss.item())

                if (i + 1) % 100 == 0:
                    #print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time() - time_now) / iter_count
                    left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
                    #print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()

                loss.backward()
                model_optim.step()

           # print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
            train_loss = np.average(train_loss)

            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f}".format(epoch + 1, train_steps, train_loss))

            adjust_learning_rate(model_optim, epoch + 1, self.args)
        path = os.path.join(self.args.checkpoints, setting)
        best_model_path = path + '/' + 'model.pth'
        torch.save(self.model.state_dict(), best_model_path)

        return self.model

    def calculate_mse(self, y_true, y_pred):
        # 均方误差
        mse = np.mean(np.abs(y_true - y_pred))
        return mse

    def predict(self, setting, load=False):
        # predict
        results = []
        preds = []

        # 加载模型
        path = os.path.join(self.args.checkpoints, setting)
        best_model_path = path + '/' + 'model.pth'
        self.model.load_state_dict(torch.load(best_model_path))

        # 评估模式
        self.model.eval()

        if args.rollingforecast:
            pre_data = pd.read_csv(args.root_path + args.rolling_data_path)
        else:
            pre_data = None

        for i in 0 if pre_data is None else range(int(len(pre_data)/args.pred_len)): 
            if i == 0:
                data_set, pred_loader = self._get_data(flag='pred')
            else:  # 进行滚动预测

                data_set, pred_loader = self._get_data(flag='pred', pre_data=pre_data.iloc[: i * args.pred_len+1])  
                print(f'滚动预测第{i + 1} 次')

            with torch.no_grad():
                for _, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(pred_loader):
                    batch_x = batch_x.float().to(self.device)
                    batch_y = batch_y.float()
                    batch_x_mark = batch_x_mark.float().to(self.device)
                    batch_y_mark = batch_y_mark.float().to(self.device)

                    dec_inp = torch.zeros([batch_y.shape[0], self.args.pred_len, batch_y.shape[2]]).float().to(
                        batch_y.device)
                    dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)

                    outputs = self.model(batch_x)
                    f_dim = -1 if self.args.features == 'MS' else 0
                    outputs = outputs[:, -self.args.pred_len:, f_dim:]
                    outputs = data_set.inverse_transform(outputs)
                    if self.args.features == 'MS':
                        for i in range(args.pred_len):
                            preds.append(outputs[0][i][outputs.shape[2] - 1])  # 取最后一个预测值即对应target列
                    else:
                        for i in range(args.pred_len):
                            preds.append(outputs[0][i][outputs.shape[2] - 1])
                    print(outputs)

        # 保存结果
        if args.rollingforecast:
            df = pd.DataFrame({'real': pre_data['{}'.format(args.target)][:len(preds)], 'forecast': preds})
            df.to_csv('./results/{}-{}-ForecastResults.csv'.format(args.data_path[:-4],args.target), index=False)
        else:
            df = pd.DataFrame({'forecast': results})
            df.to_csv('./results/{}-{}-ForecastResults.csv'.format(args.data_path[:-4],args.target), index=False)

        if args.show_results:

            # 设置绘图风格
            plt.style.use('ggplot')

            # 创建折线图
            plt.plot(df['real'].tolist(), label='real', color='blue')  # 实际值
            plt.plot(df['forecast'].tolist(), label='forecast', color='red', linestyle='--')  # 预测值

            # 增强视觉效果
            plt.grid(True)
            plt.title('real vs forecast')
            plt.xlabel('time')
            plt.ylabel('value')
            plt.legend()

            plt.savefig('results.png')

In [42]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='DLinearNet Multivariate Time Series Forecasting')
    # basic config
    parser.add_argument('--train', type=bool, default=False, help='Whether to conduct training')
    parser.add_argument('--rollingforecast', type=bool, default=True, help='rolling forecast True or False')
    parser.add_argument('--rolling_data_path', type=str, default='road20-test.csv', help='rolling data file')
    parser.add_argument('--show_results', type=bool, default=True, help='Whether show forecast and real results graph')
    parser.add_argument('--model', type=str, default='SCINet',help='Model name')

    # data loader
    parser.add_argument('--root_path', type=str, default='./data/', help='root path of the data file')
    parser.add_argument('--data_path', type=str, default='road20.csv', help='data file')
    parser.add_argument('--features', type=str, default='S',
                        help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
    parser.add_argument('--target', type=str, default='EXPONENT', help='target feature in S or MS task')
    parser.add_argument('--freq', type=str, default='5min',
                        help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
    parser.add_argument('--checkpoints', type=str, default='./models/', help='location of model models')

    # forecasting task
    parser.add_argument('--seq_len', type=int, default=288, help='input sequence length')
    parser.add_argument('--label_len', type=int, default=20, help='start token length')
    parser.add_argument('--pred_len', type=int, default=1, help='prediction sequence length')

    # model
    parser.add_argument('--individual', action='store_true', default=False,
                        help='DLinear: a linear layer for each variate(channel) individually')
    parser.add_argument('--enc_in', type=int, default=1, help='encoder input size')
    parser.add_argument('--dec_in', type=int, default=1, help='decoder input size')
    parser.add_argument('--c_out', type=int, default=1, help='output size')
    parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
    parser.add_argument('--embed', type=str, default='timeF',
                        help='time features encoding, options:[timeF, fixed, learned]')
    parser.add_argument('--activation', type=str, default='gelu', help='activation')

    # optimization
    parser.add_argument('--num_workers', type=int, default=0, help='data loader num workers')
    parser.add_argument('--train_epochs', type=int, default=20, help='train epochs')
    parser.add_argument('--batch_size', type=int, default=128, help='batch size of train input data')
    parser.add_argument('--learning_rate', type=float, default=0.001, help='optimizer learning rate')
    parser.add_argument('--loss', type=str, default='mae', help='loss function')
    parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')

    # GPU
    parser.add_argument('--use_gpu', type=bool, default=False, help='use gpu')
    parser.add_argument('--device', type=int, default=0, help='gpu')

    args = parser.parse_args(args=[])   #为了防止报错，这里换成了这个
    Exp = SCINetinitialization
    # setting record of experiments
    setting = 'predict-{}-data-{}'.format(args.model, args.data_path[:-4])

    SCI = SCINetinitialization(args)  # 实例化模型
    if args.train:
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(args.model))
        SCI.train(setting)
    print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(args.model))
    SCI.predict(setting, True)

    plt.show()

Use CPU
Number of parameters: 0.58K
>>>>>>>predicting : SCINet<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<class 'pandas.core.frame.DataFrame'>
<class 'NoneType'>
pred 1
[[[1.16569923]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第2 次
[[[1.80887993]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第3 次
[[[1.30031754]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第4 次
[[[1.77575687]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第5 次
[[[1.1757211]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第6 次
[[[1.13875223]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第7 次
[[[0.98831131]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第8 次
[[[1.06450539]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.fra

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第78 次
[[[0.90440537]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第79 次
[[[0.99712228]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第80 次
[[[1.00263166]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第81 次
[[[0.90778522]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第82 次
[[[1.02558007]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第83 次
[[[0.96546168]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第84 次
[[[0.98023042]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第85 次
[[[1.08711091]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第86 次
[[[1.05655012]]]
<class 'pa

pred 1
滚动预测第152 次
[[[2.91785343]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第153 次
[[[2.21245504]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第154 次
[[[2.07958518]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第155 次
[[[2.64766889]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第156 次
[[[2.26918645]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第157 次
[[[2.18814009]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第158 次
[[[1.9806413]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第159 次
[[[2.08194151]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第160 次
[[[2.07363145]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pr

pred 1
滚动预测第226 次
[[[5.8263686]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第227 次
[[[5.87146762]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第228 次
[[[5.1791414]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第229 次
[[[4.95817861]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第230 次
[[[5.49002392]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第231 次
[[[5.07347612]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第232 次
[[[4.91523221]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第233 次
[[[5.34633499]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred 1
滚动预测第234 次
[[[5.0085218]]]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
pred

In [43]:
import pandas as pd
import plotly.express as px
#将图像保存到本地
import plotly.io as pio
#将图像保存为SVG矢量格式
import kaleido

# 从Excel文件中读取数据
df = pd.read_csv(r'./results/{}-{}-ForecastResults.csv'.format(args.data_path[:-4],args.target))

# 取两列数据
column1 = df['real']
column2 = df['forecast']

# 对其中一列进行四舍五入保留整数操作
column_rounded = column2.round()

# 计算两列数中相等的个数
equal_count = (column_rounded == column1).sum()
print('此次预测的准确率为：',round((equal_count/288)*100,2),'%')

# 使用plotly绘制折线图
fig = px.line(df, title="Two Days Line Plot")
fig.add_scatter(x=df.index, y=column_rounded, mode='lines', name='forecast')

# 将绘图以svg格式保存到本地
#pio.write_image(fig,file='road5_prediction_results.svg', engine='kaleido', width=800, height=600)

# 显示绘图
fig.show()

此次预测的准确率为： 97.57 %
